# ***Alexanda Apostolopoulou***
# ***1115201700005***
# *Project 4 in AI (II)* 

# 2nd Model of Sentence Embeddings

In [ ]:
import pandas as pd
import numpy as np
import pickle
import glob
import json  
from pandas.io.json import json_normalize  
from nltk.tokenize import sent_tokenize
import nltk
!pip install spicy
import scipy.spatial
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

####***Note:*** I implement both questions 1 and 2 in the same notebook

## ***Step 1:*** Download and Preprocess of data

Here I download the initial dataset of COVID-19 Open Research Dataset (CORD-19) as the previous model. I selected another dataset that have 118 more articles this time.

Download the dataset

In [ ]:
# !wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-03-13.tar.gz
# !wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-03-20.tar.gz

Extract the folder

In [ ]:
# !tar -xf cord-19_2020-03-13.tar.gz
# !tar -xf cord-19_2020-03-20.tar.gz

Go to specific file

In [ ]:
# %cd /content/2020-03-13
# %cd /content/2020-03-20

Extract the desirable folder

In [ ]:
# !tar -xf comm_use_subset.tar.gz

####Here I take only the title, the abstract and the body texts of each paper and I save them to list of lists as in the previous notebook

In [ ]:
# data = []

# files = glob.glob('comm_use_subset/*', recursive=True)

# for single_file in files:
# # with open('0093f9ae0861afc0d29fff935ae6a3af898cea00.json', 'r') as f:
#   with open(single_file, 'r') as f:
#     d = json.load(f) 

#     df = pd.json_normalize(d)

#     title = df['metadata.title'].item()

#     if df['abstract'].item() == []:
#       abstract = df['abstract'].item()
#     else:
#       abstract_data = pd.json_normalize(data = d, record_path ='abstract') 
#       if (abstract_data['text'].shape[0] > 1):
#         abstract = abstract_data['text'].str.cat(sep='. ')
#       else:
#         abstract = abstract_data['text'].item()

#     text_data = pd.json_normalize(data = d, record_path ='body_text') 

#     body_list = text_data['text'].values.tolist()

#     body_list = [[el] for el in body_list] 

#     data.append([title,abstract,body_list])

In [ ]:
# print(len(data))

In [ ]:
# print(data[0])
# print('\n')
# print(data[0][0])
# print('\n')
# print(data[0][1])
# print('\n')
# print(data[0][2])
# print(len(data[0][2]))

Save the previous data locally and then in my Google Drive

In [ ]:
# import os
# cwd = os.getcwd()
# print(cwd)

In [ ]:
# with open('listfile_3.data', 'wb') as filehandle:
#     # store the data as binary data stream
#     pickle.dump(data, filehandle)

In [ ]:
# with open('listfile_3.data', 'rb') as filehandle:
#     # read the data as binary data stream
#     articles = pickle.load(filehandle)

In [ ]:
# print(len(articles))
# print(articles[0])
# print('\n')
# print(articles[0][0])
# print('\n')
# print(articles[0][1])
# print('\n')
# print(articles[0][2])

In [ ]:
# import shutil
# shutil.move('/content/listfile_3.data', '/content/drive/MyDrive/listfile_3.data')
# # shutil.move('/content/listfile_2.data', '/content/drive/MyDrive/listfile_2.data')

## ***Step 2:*** Load my preprocess data from my Drive

In [ ]:
with open('/content/drive/MyDrive/listfile_3.data', 'rb') as filehandle:
    # read the data as binary data stream
    articles = pickle.load(filehandle)

Check if everything is ok

In [ ]:
# Print the number of articles
print(len(articles))
# Print the first article
print(articles[0])
print('\n')
# Print the first title 
print(articles[0][0])
print('\n')
# Print the first abstract 
print(articles[0][1])
print('\n')
# Print the first body text 
print(articles[0][2])

9118
['Case Report Central Hypoventilation: A Case Study of Issues Associated with Travel Medicine and Respiratory Infection', "Aim. We presented the case of a child with central hypoventilation syndrome (CHS) to highlight issues that need to be considered in planning long-haul flight and problems that may arise during the flight. Case. The pediatric intensive care unit (PICU) received a child with central hypoventilation syndrome (Ondine's curse) on nocturnal ventilatory support who travelled to Hong Kong on a make-a-wish journey. He was diagnosed with central hypoventilation and had been well managed in Canada. During a long-haul aviation travel, he developed respiratory symptoms and desaturations. The child arrived in Hong Kong and his respiratory symptoms persisted. He was taken to a PICU for management. The child remained well and investigations revealed no pathogen to account for his respiratory infection. He went on with his make-a-wish journey. Conclusions. Various issues of tr

## ***Step 3:*** Tokenize sentences

####This time I selected to tokenize only the abstract of each paper, since I wanted to search for the answer firstly with criterion these parts and not in the title and the body texts. I wanted to save more time and memory.

In [ ]:
for article in range(len(articles)):
  # articles[article][0] = sent_tokenize(articles[article][0])
  if articles[article][1] != []:
    articles[article][1] = sent_tokenize(articles[article][1])
  # articles[article][2] = sent_tokenize(articles[article][2])

In [ ]:
print(articles[0][1])

['Aim.', 'We presented the case of a child with central hypoventilation syndrome (CHS) to highlight issues that need to be considered in planning long-haul flight and problems that may arise during the flight.', 'Case.', "The pediatric intensive care unit (PICU) received a child with central hypoventilation syndrome (Ondine's curse) on nocturnal ventilatory support who travelled to Hong Kong on a make-a-wish journey.", 'He was diagnosed with central hypoventilation and had been well managed in Canada.', 'During a long-haul aviation travel, he developed respiratory symptoms and desaturations.', 'The child arrived in Hong Kong and his respiratory symptoms persisted.', 'He was taken to a PICU for management.', 'The child remained well and investigations revealed no pathogen to account for his respiratory infection.', 'He went on with his make-a-wish journey.', 'Conclusions.', 'Various issues of travel medicine such as equipment, airline arrangement, in-flight ventilatory support, travel i

## ***Step 4:*** Download the Tranformers and Sentence Transformers libraries

In [ ]:
%%capture
!pip install -q sentence-transformers==0.2.5.1
!pip install -q transformers==2.5.1

## ***Step 5:*** Experiment with diferrent Pretrained Dataset



With a little Internert reasearch, I found out that there are more relevant pretrained datasets for Covid-19. So I select a model provided by gsarti for ranking purposes ***covidbert-nli***, a fine-tuned version of Deepset's CovidBERT.

This model is trained on SNLI and MultiNLI using the sentence-transformers library to produce universal sentence embeddings. Embeddings are subsequently used to perform semantic search on CORD-19.

In this notebook I follow this strategy:

- I rank the abstract of each article.
- I rank the paragraphs (body texts) of the best (most similar) abstracts.
- Comprehened the top paragraphs of the retrieved documents using HuggingFace question-answering pipeline.

For comprehension I used the CovidBERT model on SQuAD question answering data ***covid_squad***, a fine-tuned version of Deepset's CovidBERT on SQuAD dataset.

In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline, AutoModelForQuestionAnswering
from sentence_transformers import models, SentenceTransformer

I need to download and save locally the model and configure it for sentence-transformers to convert the data into embeddings.

In [ ]:
# Take specific model and its correspoding tokenizer
tokenizer = AutoTokenizer.from_pretrained('gsarti/covidbert-nli')
model = AutoModel.from_pretrained('gsarti/covidbert-nli')

# Save the model locally
model.save_pretrained('/content')
tokenizer.save_pretrained('/content')

('/content/vocab.txt',
 '/content/special_tokens_map.json',
 '/content/added_tokens.json')

In [ ]:
# Build the SentenceTransformer
word_embedding_model = models.BERT('/content')

Here I chose the pooling strategy of *mean*, since the paper recommended as the best strategy.

In [ ]:
# Add the pooling strategy of Mean
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                   pooling_mode_mean_tokens=True,
                                   pooling_mode_cls_token=False,
                                   pooling_mode_max_tokens=False)

Pass them in Sentence Tranformers 

In [ ]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Here I took the whole dataset of 9118 articles, as there isn't any problem with memory or time anymore.

In [ ]:
import copy

mini_articles = articles[:9118]

embeddings = copy.deepcopy(mini_articles)

## ***Step 6:*** Use GPU

In [ ]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available()
                      else 'cpu')

print(device)
model.to(device)

cuda:0


SentenceTransformer(
  (0): BERT(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

## ***Step 7:*** Encode Sentences 

Here I encoded only the sentences of abstracts. This was helped me to not wait so much time as in the previous notebook. The abstract content of each article is obviously much less than the content of all body texts and abstracts together.

In [ ]:
import time
start_time = time.time()

In [ ]:
origin_list_of_articles = []
list_of_articles = []

for article in embeddings:
  article_list = []

  # Choose to encode only the abstract content of each article
  abs_text = article[1]
  
  if abs_text != []:
    for sentence in abs_text:
      article_list.append(sentence)

  origin_list_of_articles.append(article_list)

  # Encode the abstract
  list_of_articles.append(model.encode(article_list))  

In [ ]:
print(len(list_of_articles))

9118


In [ ]:
end_time = time.time()
print(end_time-start_time)

372.9490256309509


## ***Step 8:*** Comprehension Model Pipeline


The *HuggingFace* provides pipeline feature for question-answering. I could provided any model and it's corresponding tokenizer, but I used covid_squad model for comprehension, as I believe that this was the closest dataset to covid data. From this model, I  will be able to take the probability associated to the answer.


In [ ]:
# device is set to -1 to use the available gpu
comprehension_model = pipeline("question-answering", model=AutoModelForQuestionAnswering.from_pretrained("graviraja/covidbert_squad"), tokenizer=AutoTokenizer.from_pretrained("graviraja/covidbert_squad"), device=-1)              

In [ ]:
print((embeddings[0][2]))

[['Travel medicine is the branch of medicine that deals with the prevention and management of health problems of international travelers [1] [2] [3] [4] . We presented the case of a child with central hypoventilation syndrome (CHS) to highlight issues that need to be considered in planning long-haul flight and problems that may arise during the flight.'], ["In the summer of 2013, the pediatric intensive care unit (PICU) of a hospital in Hong Kong received an 8-year-old boy with central hypoventilation with respiratory infection and decompensation en route to Hong Kong on a make-a-wish campaign. He was diagnosed with central hypoventilation (medullary atrophy) or Ondine's curse and had been well managed in Toronto, Canada. He was ambulatory, only needed home ventilatory support at night via tracheostomy, and inhaled salbutamol puffs on a prn basis, and he was on PEG (percutaneous endoscopic gastrostomy) feeding with puree food. Advanced fitness for air-travel arrangement was well negoti

## ***Step 9:*** Ask model for answers

Firstly, as I have the sentence embeddings of the abstract content of each article, I start by looking for the top 20 most similar articles to a given question (based only in abstracts). 

Though I retrived the top related articles, each paper contains a lot of body text content. Finding the relevant paragraphs in the article will help in narrow down the amount of data needs to comprehended for finding the answer to the given query.

After I have in my hands the top 20 articles, I started to encode the body texts of each articles. So we understant that there is a big different with the previous notebook, as here I encode the body texts of only 20 articles, while in the other notebook I encode the body texts of 9000 articles! 

I already have top 20 articles and in each document top 20 most relevant body texts. Comprehension will help even more finetuned details up to a sentence level. So, in the end, I used my comprehension model to find the appropriate answer with the biggest score.

This function returns the relative sentence of original text, given a specific paragraph (body text).

In [ ]:
import spacy

spacy_nlp = spacy.load('en_core_web_sm')

# Returns the relative sentence of original text, given a specific paragraph (body text).
def get_full_sentence(para_text, start_index, end_index):

    sent_start = 0
    sent_end = len(para_text)
    for sent in spacy_nlp(para_text).sents:
        if (sent.start_char <= start_index) and (sent.end_char >= start_index):
            sent_start = sent.start_char
        if (sent.start_char <= end_index) and (sent.end_char >= end_index):
            sent_end = sent.end_char
    sentence = para_text[sent_start:sent_end + 1]
    return sentence

In [ ]:
start_time = time.time()

In [ ]:
# Query sentences:
queries = ['What are the coronoviruses?', 
           'What was discovered in Wuhuan in December 2019?', 
           "What are common symptoms observed in COVID-19 patients?",
           'What is coronovirus Disease 2019?', 
           'What is COVID-19?', 
           'What is caused by SARS?', 
           'How is COVID-19 spread?', 
           'How does coronavirus spread?',
           "Can the use of masks prevent SARS?",
           "What is the Persistence of virus on surfaces of different materials (e.g. copper, stainless steel, plastic)?",
           "What ages have higher mortality rate in covid-19?",
           "What are the protective measures of covid-19?"
           ]

# Encode queries
query_embeddings = model.encode(queries)

# Find the closest sentences of the corpus for each query sentence based on cosine similarity and the comprehension model
for query, query_embedding in zip(queries, query_embeddings):

  # Compare all the articles' abstract content with each query
  all_abs_distances = []

  for idx_of_article,article in enumerate(list_of_articles):
    distances = []
    for idx,sentence in enumerate(article):
      distances.append((idx, (1 - (scipy.spatial.distance.cdist([query_embedding], [sentence], "cosine")[0]))))

    results = sorted(distances, key=lambda x: x[1], reverse=True)
    if results:
      all_abs_distances.append((idx_of_article, results[0][0], results[0][1]))

  results = sorted(all_abs_distances, key=lambda x: x[2], reverse=True)

  ########################################################################################
  # Take the 20 most similar articles, based on the relevant abstracts and compare all the body texts content to the query
  all_text_distances = []
  for top in results[0:20]:
    article_idx = top[0]

    body_text_distances = []
    for text_idx,text in enumerate(embeddings[article_idx][2]):
      # Encode only the body texts of 20 best articles 
      body_text_embedding = model.encode(text, show_progress_bar=False)

      body_text_distances.append(((text_idx, (1 - (scipy.spatial.distance.cdist([query_embedding], body_text_embedding, "cosine")[0])))))

    results = sorted(body_text_distances, key=lambda x: x[1], reverse=True)

    if results:
      all_text_distances.append((article_idx, results[0][0], results[0][1]))

  results = sorted(all_text_distances, key=lambda x: x[2], reverse=True)

  ########################################################################################
  # For the top 20 retrieved paragraphs in the document, answer will be comprehended on each paragraph using the model.

  answers = []
  for top_text in results[0:20]:
    article_idx = top_text[0]
    body_text_idx = top_text[1]

    query_ = {"context": embeddings[article_idx][2][body_text_idx][0], "question": query}
    pred = comprehension_model(query_, topk=1, show_progress_bar=False)

    # If there is any answer
    if pred["answer"] and round(pred["score"], 4) > 0:
        # Take the suitable sentence from the paragraph
        sent = get_full_sentence(query_['context'], pred["start"], pred["end"])
        answers.append((article_idx, round(pred["score"], 4), sent))
  ########################################################################################
  results = sorted(answers, key=lambda x: x[1], reverse=True)

  print("\n======================\n")
  print("Query:", query)
  print("\nThe most similar sentences in corpus:\n")

  if results:
    count = 1

    for res in results:
      print(count,". ", res[2], "(Score: %.4f)" % (res[1]))
      print("From the article with title: ", embeddings[res[0]][0])
      print("\n")
      if count > 3:
        break
      count += 1
  else:
    print("There isn't any answer")

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2085.68it/s]




Query: What are the coronoviruses?

The most similar sentences in corpus:

1 .  Mitochondria, lysosomes, phagosomes, Golgi complex, peroxisomes, and endoplasmic reticulum (ER) are subverted and remodeled by viruses (3, 4) , many of which use the ER as a preferred membranous compartment to build replication organelles (5) . (Score: 0.7324)
From the article with title:  Reovirus NS and NS Proteins Remodel the Endoplasmic Reticulum to Build Replication Neo-Organelles


2 .  Members exploiting ribosome shunting strategy include Cauliflower mosaic virus (CaMV), Rice tungro bacilliform virus, Sendai virus and adenovirus, etc.  (Score: 0.5875)
From the article with title:  cells Regulation of Ribosomal Proteins on Viral Infection


3 .  If so, do these viruses pose a risk for domestic animals and/or humans as observed in bat-derived severe acute respiratory syndrome (SARS), rabies, Nipah, and probably Ebola viruses? (Score: 0.1813)
From the article with title:  Expected and Unexpected Featu

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2088.80it/s]




Query: What was discovered in Wuhuan in December 2019?

The most similar sentences in corpus:

1 .  In December 2019, China detected many cases of viral pneumonia-like disease similar to SARS that were confirmed to be caused by novel Betacoronavirus, provisionally called 2019 novel coronavirus (2019-nCoV).  (Score: 0.1037)
From the article with title:  pathogens Emergence of Novel Coronavirus 2019-nCoV: Need for Rapid Vaccine and Biologics Development


2 .  Since the announcement of a cluster of pneumonia cases of unknown etiology in Wuhan, Hubei Province, China, was made on 31 December 2019, many rapid virological, clinical, and epidemiological research responses have taken place [1, 2] .  (Score: 0.0001)
From the article with title:  Clinical Medicine The Extent of Transmission of Novel Coronavirus in




add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2109.81it/s]


...



Query: What is the Persistence of virus on surfaces of different materials (e.g. copper, stainless steel, plastic)?

The most similar sentences in corpus:

1 .  The NA proteins of human FLUAV facilitate release of progeny particles from infected cells by removing sialic acids from the cell surface.  (Score: 0.0948)
From the article with title:  The Hemagglutinin of Bat-Associated Influenza Viruses Is Activated by TMPRSS2 for pH-Dependent Entry into Bat but Not Human Cells


2 .  Diverse phenotypes of ZIKV have been demonstrated [41, 54, 55] where African strains showed a higher infection rate than Asian strains, and varying yields of virus produced in human neuronal cell lines were recently reported.  (Score: 0.0848)
From the article with title:  Mitigating the risk of Zika virus contamination of raw materials and cell lines in the manufacture of biologicals


3 .  The model considers spleen, liver and blood compartments, in which MHV replication is described at different resolution 

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1854.25it/s]




Query: What ages have higher mortality rate in covid-19?

The most similar sentences in corpus:

1 .  Nearly half of the patients (8/18) had hypertension, two had diabetes, three had coronary heart diseases, and three had COPD, indicating that chronic heart and lung diseases would increase risk for H7N9 infection. (Score: 0.1372)
From the article with title:  Clinical Features and Factors Associated with Outcomes of Patients Infected with a Novel Influenza A (H7N9) Virus: A Preliminary Study


2 .  : Flu+, 0.41 (0.24-0.69); Flu-, 0.42 (0.20, 0.90). (Score: 0.0012)
From the article with title:  Gastroenteritis and respiratory infection outbreaks in French nursing homes from 2007 to 2018: Morbidity and all-cause lethality according to the individual characteristics of residents


3 .  One-way ANOVA analysis was conducted to determine the age-related risk of BPH, OAB, and QoL. The risk of BPH significantly increased with age (12.65±7.96 for those aged 60-69 years and 15.21±8.86 for thos

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1961.79it/s]




Query: What are the protective measures of covid-19?

The most similar sentences in corpus:

1 .  To reduce transmission of infectious diseases. (Score: 0.5282)
From the article with title:  BMC Public Health Applying the balanced scorecard to local public health performance measurement: deliberations and decisions


2 .  Following SARS, there were a number of risk communication technical assistance activities primarily coordinated by U.S. CDC, the WHO, and its Western Pacific Regional Office (WPRO).  (Score: 0.0633)
From the article with title:  


3 .  A P value of less than 0.05 was considered significant. (Score: 0.0189)
From the article with title:  Sublingual immunization with recombinant adenovirus encoding SARS-CoV spike protein induces systemic and mucosal immunity without redirection of the virus to the brain


4 .  In the prophylaxis scenario, we included adherence to prophylaxis and incidence of adverse events  (Score: 0.0159)
From the article with title:  Cost-Effectiven

In [ ]:
end_time = time.time()
print(end_time-start_time)

498.7024712562561


## ***Step 10:*** Summaries

- In general, I have to admit that I was more satisfied with my second model. The answers that it gives are more relevant and right (in most cases) than the first pretrained model. This might happens for these reasons:
  - For the second model I used the pretrained model of covidbert-nli dataset and the comprehension model with pipeline from transformers with the pretrained model for question answering of covidbert_squad, instead of pretrained nli-roberta-base that I used in the my first model. First model is trained on general data, but the second model is based in data that concern the covid entirely. So it is very logic to me to give a better performance in questions that concern only the covid issues.
  - If we look carefully, we will see that the second model, never returns me a answer that it is only one word like the first does. The first model returned me answers like "(IM19)" or "(DOCX) S19 Information" or "1.19)." or "19 )." many times. This improvement may be due to the fact that I used the complehension model and I search for the sentence-answer in a paragraph by myself and not tokenized all of my data.
- In many questions the answers is the requested from the very first answer. For instance, in the question "What was discovered in Wuhuan in December 2019?" the given answers were "In December 2019, China detected many cases of viral pneumonia-like disease similar to SARS that were confirmed to be caused by novel Betacoronavirus, provisionally called 2019 novel coronavirus (2019-nCoV)." and "Since the announcement of a cluster of pneumonia cases of unknown etiology in Wuhan, Hubei Province, China, was made on 31 December 2019, many rapid virological, clinical, and epidemiological research responses have taken place.", which both of them are correct. Moreover, for the question "What is coronovirus Disease 2019?" the answer is "The causative agent of the pneumonia is suggested to be a novel coronavirus (2019-nCoV) of the same lineage (but genetically distinct) from the coronavirus causing severe acute respiratory syndrome (SARS)" and the question "How is COVID-19 spread?" the answer is "SARS-CoV is thought to be transmitted by respiratory droplets produced when an infected person coughs or sneezes." which are very satisfactory. The first model couldn't answer to these questions right and even more so from the first answer.
- At some point, I notice that for many questions that includes the word "covid-19" has irrelevant answers, both in the first and the second notebook. I discovered after printing the top 15 abstracts for these questions, was in the above form:
    - Conceptualization: PZ SP. 
    - 19.
    - 2019; 21(4): e12251.
    - IgA responses.
    - Trends Parasitol.
    - 2019; 11(2): pii: E176.
    - mSphere 5:e00807-19.
    - The expression of R 0 is as follows:
    - a, the).
    - (Fam.
    - (A) Estimates of R 0 .
    - S2) .
    - or i.m.
    - In Exp.
    - In exp.

  This is surely caused by the tokenization of abstracts context that had more complex texts and the sentences was splitted to the previous form. This is something that I couldn't control by myself, but it explains the reason that some of the answers have many low score and they are not very relevant to their questions. These abstracts (and these articles in general) aren't representative for looking to them the answer.
- Sometimes the model may returns two times the same answer. This is because this sentence may be twice in the bodytexts of the aricles and it isn't something that we should worried about.
- There are some asnwers that even if they are very relative and representative to their question, the given score isn't so good. This may happens because this model doesn't use cosine similarity as a metric to see how similar is the texts. Futhermore, the data that managed were very few. I did comparisons between only 20 paragraphs! So I was very satisfied if with so few data gives me these results.
- The above comment may is responsible for some answers aren't always 4, but fewer.


### ***Compare your 2 models based on at least 2 different criteria of your choice:***

The main criteria that I select to compare my models is the ***time*** and the ***computing power and memory***:
- In the first model I used all the articles with all of their abstracts and body texts. In general I applied the brute force method in all of my data. Something like that is needed a lot of computer power and memory. Sometimes, I have to mention, in my tests I faced memory crashed and it took many hours to find a way to deal with it. If I didn't use GPU the first notebook would needed countless hours to run and it isn't sure that my memory will allow it. On the other hand, in the second notebook I used only the abstracts of the articles, which they weren't so much in quantity and in the end I used only the body texts from the 20 most similar abstracts. Undoubtedly, I used much less data in the second notebook and I take more satisfying answers. Moreover, because I used less data, I didn't need so much the GPU. I could run it without it and it will take surely more time but I would not have any problem with computing power or memory like the first notebook that it doesn't run without the usage of GPU cuda.
- Another vital criterion is surely the time that the 2 notebooks needed to run. In the first notebook, the "encode section" took ~1 hour to run and the "question-answering search section" took just one second. So the total time of 1st notebook is about 1 hour. The 2nd notebook took ~6 minutes for the "encode section" and ~8 minutes for the "question-answering search section". So the total time of the 2nd notebook is maximum 15 minutes. It is a big time difference, given the fact that the second notebook with less data and time gives more satisfactory answers. This is a big improvement!
- From the above, we understand that I based mainly also in the accuracy of my results. The relation between ***time and accuracy*** is a serious criterion that I took into account. And in this fact it helped a lot that the pretrained datasets in the second notebook had immediate relation with the covid. Besides, the comprehendion model that uses question-answering pipeline from transformers library contributed equally. If the second model took less time but more inaccurate results, I wouldn't choose it.
- Last criterion I have to refer is the ***metric*** I used to take this results. In the first notebook I used the method of textual similarity based completely in cosine similarity and it returns the sentences that their words are more similar to the question. So it is very logic to returns the most similar sentence or word and not the answer. In the second notebook I used the metric of cosine similarity to take the top 20 similar abstracts and paragraphs, but in the end I used the prediction that the pipeline comprehension model of question-answering returns to me.
